In [3]:
# Install CUDA C++ plugin for Colab:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpm54m5tr5".


In [4]:
# Detect selected GPU and its NVIDA architecture:
import subprocess
gpu_info = subprocess.getoutput("nvidia-smi --query-gpu=name,compute_cap --format=csv,noheader,nounits")
if "not found" in gpu_info.lower(): raise RuntimeError("Error: No GPU found. Please select a GPU runtime environment.")
gpu_name, compute_cap = map(str.strip, gpu_info.split(','))
gpu_arch = f"sm_{compute_cap.replace('.', '')}"

print(f"{'GPU Name':<15}: {gpu_name}")
print(f"{'Architecture':<15}: {gpu_arch}")

GPU Name       : Tesla T4
Architecture   : sm_75


In [8]:
%%cuda -c "--gpu-architecture $gpu_arch"
#include <stdio.h>

__global__ void first_kernel(){
    // blockDim(x,y,z) number of threads in the block
    // blockIdx(x,y,z) index of block in the grid
    // threadIdx(x,y,z) index of thread in the block
    // globalId is the thread id globally
    printf("blockDim = (%u, %u, %u)\n",blockDim.x, blockDim.y, blockDim.z);
    int blockId = blockIdx.x;
    int threadId = threadIdx.x;
    int globalId = threadId + blockId * blockDim.x;
    printf("Block id %d , Thread id in the block %d  Global thread id %d\n", blockId, threadId, globalId);
}
int main(){
    int numBlocks=4;
    int threadsPerBlock=8;
    first_kernel<<<numBlocks, threadsPerBlock>>>();
     cudaDeviceSynchronize();
    return 0;
}

blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
blockDim = (8, 1, 1)
Block id 2 , Thread id in the block 0  Global thread id 16
Block id 2 , Thread id in the block 1  Global thread id 17
Block id 2 , Thread id in the block 2  Global thread id 18
Block id 2 , Thread id in the block 3  Global thread id 19
Block id 2 , Thread id in the block 4  Global thread id 20
Block id 2 , Thread id in the blo

UsageError: Cell magic `%%cuda` not found.
